In [142]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

In [17]:
"""
connect to db
"""
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)

In [26]:
cluster = pd.read_excel('Cluster_Lin.xlsx')
cluster = pd.concat([
    cluster.loc[:, ['group',1]], 
    cluster.loc[:, ['group',2]], 
    cluster.loc[:, ['group',3]], 
    cluster.loc[:, ['group',4]],
    cluster.loc[:, ['group',5]]], axis=0).fillna(method='ffill', axis=1)
cluster = cluster[cluster.loc[:, 5] > 999].loc[:, ['group',5]].sort_values(['group', 5]).rename({5:'代號', 'group':'歆凱分組'}, axis=1)
cluster = pd.merge(cluster, pd.read_excel('stock_info.xls', sheet_name='產業別').loc[
    :, ['代號','名稱','交易所主產業代碼','交易所子產業代碼','TEJ主產業代碼','TEJ子產業代碼']], on='代號', how='left')
cluster['TEJ主產業代碼'] = cluster['TEJ主產業代碼'].astype(str)
cluster['代號']= cluster['代號'].astype(int)

new_industry = pd.read_excel('stock_info.xls', sheet_name='主產業').drop('產業英文簡稱', axis=1)
new_industry['TEJ主產業代碼'] = new_industry['TEJ主產業代碼'].str.strip().str[1:]
minor_industry = pd.read_excel('stock_info.xls', sheet_name='子產業').drop('產業英文簡稱', axis=1)
minor_industry['TEJ子產業代碼'] = minor_industry['TEJ子產業代碼'].str.strip().str[1:]

cluster = pd.merge(cluster, new_industry, on='TEJ主產業代碼', how='left')
cluster = pd.merge(cluster, minor_industry, on='TEJ子產業代碼', how='left')
cluster['industry_title'] = np.where(cluster.minor_industry.isna() == True, 
                                     cluster.new_industry + '類', 
                                     cluster.new_industry + ':' + cluster.minor_industry + '類')
cluster.isna().sum()

In [11]:
daily_trading = pd.concat([
    pd.read_excel('220207_daily.xlsx'), 
    pd.read_excel('220901_daily.xlsx'), 
    pd.read_excel('210901_daily.xlsx')
], axis=0)
daily_trading = daily_trading[daily_trading['代號'] <= '9999'].sort_values(['代號', '年月日'])
daily_trading = daily_trading.rename({
    '代號':'st_code', '名稱':'st_name', '年月日':'date', 
    '開盤價(元)':'Open', '最高價(元)':'High', '最低價(元)':'Low', '收盤價(元)':'Close'
}, axis=1)

### TEJ 產業分類

In [290]:
industry = pd.read_excel('stock_info.xls', sheet_name='產業別').loc[
    :, ['代號','名稱','交易所主產業代碼','交易所子產業代碼','TEJ主產業代碼','TEJ子產業代碼']]

new_industry = pd.read_excel('stock_info.xls', sheet_name='主產業').drop('產業英文簡稱', axis=1)
new_industry['TEJ主產業代碼'] = new_industry['TEJ主產業代碼'].str.strip().str[1:]
minor_industry = pd.read_excel('stock_info.xls', sheet_name='子產業').drop('產業英文簡稱', axis=1)
minor_industry['TEJ子產業代碼'] = minor_industry['TEJ子產業代碼'].str.strip().str[1:]

industry = pd.merge(industry, new_industry, on='TEJ主產業代碼', how='left')
industry = pd.merge(industry, minor_industry, on='TEJ子產業代碼', how='left')

industry['industry_title'] = np.where((industry.minor_industry.isna() == True) 
                                      & (industry.new_industry.isna() == False), 
                                     industry.new_industry + '類', 
                                      np.where((industry.minor_industry.isna() == False) 
                                               & (industry.new_industry.isna() == True), 
                                               industry.minor_industry + '類', 
                                               industry.new_industry + ':' + industry.minor_industry + '類'))
industry = industry.loc[:, ['代號', '名稱', 'new_industry', 'minor_industry','industry_title']]

#### 搜尋系統

In [130]:
def latest_return(data):
    data['近一日漲跌%'] = round(data.Close.pct_change(1), 4)
    data['近一週漲跌%'] = round(data.Close.pct_change(5), 4)
    data['近一月漲跌%'] = round(data.Close.pct_change(20), 4)
    return data

In [301]:
# ppl give a stock number
st_input = '2330'

try:
    st_group_num = cluster[cluster['代號'] == st_input]['歆凱分組'].values[0]
    gruop_num = cluster[cluster['歆凱分組'] == st_group_num]
    group_data = daily_trading[daily_trading.st_code.isin(gruop_num['代號']) == True]
    group_title = 'TEJ為 '+['、'.join(gruop_num.new_industry.drop_duplicates().to_list())][0] + ' 概念股報酬走勢'

except IndexError:
    print('跳通知：ML並無幫他分類到組別')
    st_group_num = industry[industry['代號'] == int(st_input)]['new_industry'].values[0]
    gruop_num = industry[industry['new_industry'] == st_group_num]
    group_data = daily_trading[daily_trading.st_code.isin(gruop_num['代號'].astype(str)) == True]
    group_title = 'ML並無分類，在TEJ為 '+['、'.join(gruop_num.new_industry.drop_duplicates().to_list())][0] + ' 概念股報酬走勢'
    
    
group_data = group_data.groupby('st_code', as_index=False).apply(latest_return)
group_data.loc[:, ['近一日漲跌%', '近一週漲跌%', '近一月漲跌%']] = group_data.loc[:, ['近一日漲跌%', '近一週漲跌%', '近一月漲跌%']]*100

latest = group_data.loc[:, ['st_code', 'st_name','近一日漲跌%', '近一週漲跌%', '近一月漲跌%']]
latest = latest.drop_duplicates('st_code', keep='last').reset_index(drop=True)
group_return = group_data.groupby('date', as_index=False).agg(
    {'近一日漲跌%':'mean', '近一週漲跌%':'mean', '近一月漲跌%':'mean'})
group_return['color'] = np.where(group_return['近一日漲跌%']>0, 'pink', 'green')
group_return['近一日漲跌%'] = group_return['近一日漲跌%'].cumsum()
group_return_figure = group_return.tail(1)
group_return_figure

# 族群報酬率圖
mask = group_return['近一日漲跌%'] >= 0
group_return['ret_above'] = np.where(mask, group_return['近一日漲跌%'], 0)
group_return['ret_below'] = np.where(mask, 0, group_return['近一日漲跌%'])

fig = go.Figure(go.Scatter(
    y=group_return['ret_above'],
    x=group_return.date, 
    fill='tozeroy',
    mode='none',
    text=group_return['近一日漲跌%']))

fig.add_trace(go.Scatter(
    y=group_return['ret_below'],
    x=group_return.date, 
    fill='tozeroy',
    mode='none',
    text=group_return['近一日漲跌%']))

fig.update_xaxes(showspikes=True, spikecolor="rgb(204, 204, 204)", spikemode="across", spikethickness=0.1)
fig.update_layout(
#     hovermode='x',
    title = group_title,
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(204, 204, 204)',
        gridcolor='rgb(204, 204, 204)',
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'

)
fig.update_xaxes(title='日期')
fig.show()

# 相關關鍵字：
related_keyword = gruop_num['new_industry'].drop_duplicates().to_list() + gruop_num['minor_industry'].drop_duplicates().to_list()
print(group_return_figure)
print(related_keyword)
print(latest)


['電子零組件', '半導體', '電源供應器', '晶圓代工', 'IC設計', 'IC封測']
  st_code st_name  近一日漲跌%  近一週漲跌%  近一月漲跌%
0    2308     台達電   -1.21   -2.72    0.17
1    2330     台積電   -1.35   -1.35    1.59
2    2454     聯發科   -2.95    0.00    4.33
3    3711   日月光投控    2.42    3.92    4.43
4    5347      世界   -0.96    0.11    0.54
          date  近一日漲跌%  近一週漲跌%  近一月漲跌%  color
360 2023-02-24 -11.832  -0.008   2.212  green


'金融控股'

In [273]:
industry.sort_values('交易所主產業代碼')['交易所主產業代碼'].unique()
# daily_trading

array(['M1100', 'M1200', 'M1300', 'M1400', 'M1500', 'M1600', 'M1700',
       'M1800', 'M1900', 'M2000', 'M2100', 'M2200', 'M2300', 'M2500',
       'M2600', 'M2700', 'M2800', 'M2900', 'M3200', 'M3300', 'M3400',
       'M9700', 'M9900'], dtype=object)

M1100:水泥工業
M1200:食品工業
M1300:塑膠工業
M1400:紡織工業
M1500:電機機械
M1600:電器電纜
M1700:化學生技醫療
M1800:玻璃陶瓷
M1900:造紙工業
M2000:鋼鐵工業
M2100:橡膠工業
M2200:汽車工業
M2300:電子工業
M2500:建材營造
M2600 航運業
M2700:觀光
M2800:金融
M2800:金融保險
M2900:百貨
M3100:投資信託
M3300:農業科技
M3400:電子商務
M9800:綜合
M9700:油電燃氣
M9900:其他